In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import random as random

In [2]:
df_train=pd.read_csv('mnist_train.csv')
df_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X_train=df_train.drop('label', axis=1)
Y_train=df_train['label']

In [4]:
#scaler=StandardScaler()
#X=scaler.fit_transform(X_temp)
for i in X_train.columns:
    if X_train[i].max()-X_train[i].min()==0:
        normalized_col=X_train[i]*0
    else:
        normalized_col = (2*(X_train[i]-X_train[i].min())/(X_train[i].max()-X_train[i].min())) - 1
    X_train[i]=normalized_col
#X_train,X_test,Y_train,Y_test=train_test_split(X_temp,Y_temp,test_size=0.2)
#print(X_temp)

In [5]:
df_test=pd.read_csv('mnist_test.csv')
df_test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_test=df_test.drop('label', axis=1)
Y_test=df_test['label']

In [7]:
for i in X_test.columns:
    if X_test[i].max()-X_test[i].min()==0:
        normalized_col=X_test[i]*0
    else:
        normalized_col = (2*(X_test[i]-X_test[i].min())/(X_test[i].max()-X_test[i].min())) - 1
    X_test[i]=normalized_col

In [8]:
class SVM_multi():
    
    #initiating hyper-parameters
    def __init__(self, l_rt, n_iter, lamda_w, lamda_e, pred_num, algo, sample):
        self.l_rt=l_rt
        self.n_iter=n_iter
        self.lamda_w=lamda_w
        self.lamda_e=lamda_e
        self.pred_num=pred_num
        self.algo=algo
        self.sample=sample
        
    #fitting training data
    def fit(self,X,Y):
        self.row=X.shape[0]
        self.col=X.shape[1]
        self.wt=np.zeros(self.col)
        self.b=0
        self.X=X
        self.Y=Y
        
        for i in range(self.n_iter):
            self.modify_wt()
    
    #grad descend
    def modify_wt(self):
        Y_label=np.where(self.Y!=self.pred_num,-1,1)
        if self.algo=='gradient_descend':
            for i, x in enumerate(self.X):
                if 1 - Y_label[i]*(np.dot(self.wt,x)+self.b) <=0 :
                    dw = 2*self.lamda_w*self.wt
                    db = 0
                else:
                    dw = (2*self.lamda_w*self.wt) - self.lamda_e*Y_label[i]*x
                    db = -Y_label[i]
                self.wt = self.wt - (self.l_rt*dw)
        if self.algo=='stochastic_descend':
            x_rand = random.sample(list(self.X.values), self.sample)
            for i, x in enumerate(x_rand):
                if 1 - Y_label[i]*(np.dot(x,self.wt)+self.b) <=0 :
                    dw = 2*self.lamda_w*self.wt
                    db = 0
                else:
                    dw = (2*self.lamda_w*self.wt) - self.lamda_e*Y_label[i]*x
                    db = -Y_label[i]
                self.wt = self.wt - (self.l_rt*dw)
                self.b=self.b - (self.l_rt*db)
            
            
    #predicting test data
    def predict(self,X):
        result= np.dot(X,self.wt)
        predicted_result=np.sign(result)
        predicted_labels=np.where(predicted_result <=-1,0,1)
        dist_from_plane=result/np.dot(self.wt,self.wt)
        return predicted_labels,dist_from_plane
    
    
    def accuracy_score(self,Y_test,Y_pred):
        if len(Y_test)==len(Y_pred):
            true_count=0
            for i in range(len(Y_test)):
                if Y_test[i]==Y_pred[i]:
                    true_count+=1
            return true_count/len(Y_test)
        else:
            print('lenght of both arrays are different')
            
            
    def recall_binary(self,Y_test,Y_pred):
        self.Y_test=Y_test
        self.Y_pred=Y_pred
        if len(self.Y_test)==len(self.Y_pred):
            true_pos=0
            actual_pos=0
            for i in range(len(self.Y_test)):
                if self.Y_test[i]==1:
                    actual_pos+=1
                if self.Y_test[i]==self.Y_pred[i] and self.Y_test[i]==1:
                    true_pos+=1
            print('true positive = ',true_pos,'total positive = ',actual_pos)
            return true_pos/actual_pos
        else:
            print('lenght of both arrays are different')
            
                    
        
        
    
                

In [9]:
for i in range(10):
    svm=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=1, lamda_e=10, pred_num=i, algo='stochastic_descend', sample=10)
    svm.fit(X_train,Y_train)
    Y_train_pred,d_1=svm.predict(X_train)
    Y_test_pred,d_2=svm.predict(X_test)
    temp_train=np.where(Y_train!=i,0,1)
    temp_test=np.where(Y_test!=i,0,1)
    accuracy_train_score=svm.accuracy_score(temp_train,Y_train_pred)
    accuracy_test_score=svm.accuracy_score(temp_test,Y_test_pred)
    recall_train=svm.recall_binary(temp_train,Y_train_pred)
    recall_test=svm.recall_binary(temp_test,Y_test_pred)
    
    print('predicted digit = ',i, ' training data accuracy = ',accuracy_train_score, ' testing data accuracy = ',accuracy_test_score, ' training data recall = ',recall_train, ' testing data recall = ',recall_test)
    print('**********')

true positive =  0 total positive =  5923
true positive =  0 total positive =  980
predicted digit =  0  training data accuracy =  0.89785  testing data accuracy =  0.891  training data recall =  0.0  testing data recall =  0.0
**********
true positive =  0 total positive =  6742
true positive =  0 total positive =  1135
predicted digit =  1  training data accuracy =  0.86345  testing data accuracy =  0.8584  training data recall =  0.0  testing data recall =  0.0
**********
true positive =  211 total positive =  5958
true positive =  64 total positive =  1032
predicted digit =  2  training data accuracy =  0.8846333333333334  testing data accuracy =  0.8754  training data recall =  0.035414568647197046  testing data recall =  0.06201550387596899
**********
true positive =  0 total positive =  6131
true positive =  0 total positive =  1010
predicted digit =  3  training data accuracy =  0.8976666666666666  testing data accuracy =  0.8986  training data recall =  0.0  testing data recal

In [ ]:
svm_0=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=0, algo='stochastic_descend', sample=10)
svm_1=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=1, algo='stochastic_descend', sample=10)
svm_2=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=2, algo='stochastic_descend', sample=10)
svm_3=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=3, algo='stochastic_descend', sample=10)
svm_4=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=4, algo='stochastic_descend', sample=10)
svm_5=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=5, algo='stochastic_descend', sample=10)
svm_6=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=6, algo='stochastic_descend', sample=10)
svm_7=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=7, algo='stochastic_descend', sample=10)
svm_8=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=8, algo='stochastic_descend', sample=10)
svm_9=SVM_multi(l_rt=0.01, n_iter=1000, lamda_w=0.001, lamda_e=100, pred_num=9, algo='stochastic_descend', sample=10)

In [ ]:
svm_0.fit(X_train,Y_train)
svm_1.fit(X_train,Y_train)
svm_2.fit(X_train,Y_train)
svm_3.fit(X_train,Y_train)
svm_4.fit(X_train,Y_train)
svm_5.fit(X_train,Y_train)
svm_6.fit(X_train,Y_train)
svm_7.fit(X_train,Y_train)
svm_8.fit(X_train,Y_train)
svm_9.fit(X_train,Y_train)

In [ ]:
final_pred=[]
count_0=0
count_1=0
count_more=0
y_0_pred=[]
y_1_pred=[]
y_2_pred=[]
y_3_pred=[]
y_4_pred=[]
y_5_pred=[]
y_6_pred=[]
y_7_pred=[]
y_8_pred=[]
y_9_pred=[]
for x in X_test.values:
    dist_from_plane=[]
    temp_dist=[]
    labels=[]
    
    Y_test_pred,dist=svm_0.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_0_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_1.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_1_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_2.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_2_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_3.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_3_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_4.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_4_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_5.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_5_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_6.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_6_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_7.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_7_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_8.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_8_pred.append(int(Y_test_pred))
    
    Y_test_pred,dist=svm_9.predict(x)
    dist_from_plane.append(dist)
    labels.append(int(Y_test_pred))
    y_9_pred.append(int(Y_test_pred))
    
    for i in range(10):
        count=0
        if labels[i]==1:
            count+=1
        if dist_from_plane[i]<=0:
                dist_from_plane[i]=abs(dist_from_plane[i])
    
    if count==0:
        final_pred.append(dist_from_plane.index(min(dist_from_plane)))
        count_0+=1
    elif count==1:
        final_pred.append(labels.index(1))
        count_1+=1
    else:
        for i in range(10):
            if labels[i]==1:
                temp_dist.append(dist_from_plane[i])
        final_pred.append(dist_from_plane.index(max(temp_dist)))
        count_more+=1 

In [ ]:
accuracy_test_score=svm_0.accuracy_score(Y_test.values,final_pred)
print(accuracy_test_score)
print(count_0,count_1,count_more)


In [ ]:
compiled_svm_pred=[y_0_pred,y_1_pred,y_2_pred,y_3_pred,y_4_pred,y_5_pred,y_6_pred,y_7_pred,y_8_pred,y_9_pred]
recall=[]
for i in range(10):
    temp_Y_test=np.where(Y_test!=i,0,1)
    recall_test=svm.recall_binary(temp_Y_test,compiled_svm_pred[i])
    recall.append(recall_test)
final_recall=np.array(recall).mean()
print('recall =',final_recall)